In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings

# Filter out specific warning by category
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import tensorflow as tf
import os

In [ ]:
# Image size and batch size
image_size = (380, 380)
batch_size = 64

In [ ]:
# The main folder
base_path = '/kaggle/input/images'

# Create list of file paths and corresponding labels
file_paths = []
labels = []

# Iterate through 'fake' and 'real' subfolders to collect file paths and labels
for label, category in enumerate(['fake', 'real']):
    category_path = os.path.join(base_path, category)
    for file_name in os.listdir(category_path):
        file_path = os.path.join(category_path, file_name)
        file_paths.append(file_path)
        labels.append(label)  # Assign label 0 for 'fake' and 1 for 'real'

In [ ]:
import pandas as pd

# Check class distribution
class_distribution = pd.Series(labels).value_counts()
print("Class Distribution:")
print(class_distribution)

# Calculate class percentages
total_samples = len(labels)
class_percentages = class_distribution / total_samples
print("\nClass Percentages:")
print(class_percentages)

In [ ]:
import cv2
# Check the dimension of the data
def get_image_dimensions(file_path):
    img = cv2.imread(file_path)

    if img is not None:
        # Get image dimensions (height, width, channels)
        height, width, channels = img.shape
        return height, width, channels
    else:
        print(f"Error: Failed to load image at {file_path}")
        return None

# Display dimensions of the first 5 images
num_images_to_display = 5

if len(file_paths) >= num_images_to_display:
    print(f"Displaying dimensions of the first {num_images_to_display} images:")

    for i in range(num_images_to_display):
        image_path = file_paths[i]
        dimensions = get_image_dimensions(image_path)

        if dimensions is not None:
            height, width, channels = dimensions
            print(f"Image {i+1}:")
            print(f"  Height: {height} pixels")
            print(f"  Width: {width} pixels")
            print(f"  Channels (depth): {channels}")
        else:
            print(f"Image {i+1}: Error loading image dimensions")
else:
    print("No image files found")

In [ ]:
import matplotlib.pyplot as plt
import random

num_images_to_display = 5

if len(file_paths) >= num_images_to_display:
    plt.figure(figsize=(15, 8))
    random_indices = random.sample(range(len(file_paths)), num_images_to_display)

    for i, idx in enumerate(random_indices):
        image_path = file_paths[idx]
        label = labels[idx]
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for matplotlib

        # Display image with label
        plt.subplot(1, num_images_to_display, i + 1)
        plt.imshow(img)

        if label == 0:
            plt.title("Fake")
        elif label == 1:
            plt.title("Real")

        plt.axis('off')

    plt.show()
else:
    print("No image files found")

In [ ]:
img_augmentation_layers = [
    tf.keras.layers.RandomRotation(factor=0.15),
    tf.keras.layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomContrast(factor=0.1)
]

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.applications.efficientnet import preprocess_input

# Function to preprocess the data with optional data augmentation and manual normalization
def preprocess_image(file_path, label, augment=False, image_size=(380, 380)):
    # Load and decode image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Ensure 3 channels (RGB)

    # Apply image augmentation if specified
    if augment:
        for layer in img_augmentation_layers:
            img = layer(img)


    # Resize image to the required input size for EfficientNetB4
    img = tf.image.resize(img, image_size)


    # Preprocess image using EfficientNet preprocessing
    img = preprocess_input(img)

    return img, label

In [ ]:
from sklearn.model_selection import train_test_split

# Split data into training, validation, and test sets with stratified sampling
train_file_paths, test_file_paths, train_labels, test_labels = train_test_split(file_paths, labels, test_size=0.2, stratify=labels, random_state=42)

train_file_paths, val_file_paths, train_labels, val_labels = train_test_split(train_file_paths, train_labels, test_size=0.25, stratify=train_labels, random_state=42)

# Print the length
print("Number of training samples:", len(train_file_paths))
print("Number of training labels:", len(train_labels))
print("Number of validation samples:", len(val_file_paths))
print("Number of validation labels:", len(val_labels))
print("Number of test samples:", len(test_file_paths))
print("Number of test labels:", len(test_labels))

In [ ]:
from keras.utils import is_keras_tensor
import tensorflow as tf

x = tf.keras.Input(shape=(10,))
print(is_keras_tensor(x))  # Should print True

In [ ]:
# Create TensorFlow datasets for training, validation, and test
#rom tensorflow.keras.engine.keras_tensor import KerasTensor


train_dataset = tf.data.Dataset.from_tensor_slices((train_file_paths, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_file_paths, val_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_file_paths, test_labels))

# Map preprocessing function to the datasets
# Only perform data augmentation on Train dataset
train_dataset = train_dataset.map(lambda x, y: preprocess_image(x, y, augment=True), num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.map(lambda x, y: preprocess_image(x, y, augment=False), num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(lambda x, y: preprocess_image(x, y, augment=False), num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle and batch the training dataset
train_dataset = train_dataset.shuffle(buffer_size=len(train_file_paths)).batch(batch_size)

# Batch the validation dataset. I wont shuffle and perform data augmentation on  test and vad
val_dataset = val_dataset.batch(batch_size)

# Batch the test dataset
test_dataset = test_dataset.batch(batch_size)

In [ ]:
# # Function to display preprocessed image
# def show_preprocessed_images(dataset, num_images=5):
#     plt.figure(figsize=(15, 10))
#     for i, (image, label) in enumerate(dataset.take(1)): # only 1 batch
#         for j in range(num_images):
#             plt.subplot(1, num_images, j + 1)
#             plt.imshow(image[j])  # Display j-th image in the batch
#             plt.title(f"Label: {label[j].numpy()}")
#             plt.axis("off")
#     plt.show()
# # Visualize preprocessed images from the train dataset
# print("Visualizing preprocessed images from the train dataset:")
# show_preprocessed_images(train_dataset)

In [ ]:
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Build the base model architecture
base_model = EfficientNetB4(
    include_top=False,
    weights=None,  # Don't load imagenet weights, we'll load your .h5 weights
    input_shape=(image_size[0], image_size[1], 3)
)

# Load weights into the base model
base_model.load_weights('/kaggle/input/efficientnetb4/tensorflow2/default/1/efficientnetb4_notop.h5')

# Freeze the base model layers
base_model.trainable = False

# Add classification head
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.001))
])

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

# Model summary
model.summary()

In [ ]:
# Train the model with fine-tuning
epochs = 5
model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[reduce_lr])

In [ ]:
# Unfreeze the last 20 layers of the base model
fine_tune_at = -20

# Unfreeze the selected layers for fine-tuning
for layer in base_model.layers[fine_tune_at:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# Specify a lower learning rate for the fine-tuned layers
optimizer = Adam(learning_rate=0.0001)  # Lower learning rate for fine-tuned layers
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Set up a learning rate reduction scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# ModelCheckpoint callback to save the best model based on validation loss
checkpoint_callback = ModelCheckpoint(
    filepath='best_model_fc.keras',
    monitor='val_loss',  # Monitor validation loss
    save_best_only=True,  # Save only the best model
    mode='min',  # Save the model with the lowest validation loss
    verbose=1
)

In [ ]:
# Train the model with fine-tuning
epochs = 5   
model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[reduce_lr , checkpoint_callback])

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Predictions on the test dataset
predictions = model.predict(test_dataset)
predicted_labels = (predictions > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)

# Convert labels from dataset to numpy array for evaluation
true_labels = np.array(test_labels)

# Compute confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Calculate precision, recall, and F1 score
precision = cm[1, 1] / (cm[1, 1] + cm[0, 1])  # TP / (TP + FP)
recall = cm[1, 1] / (cm[1, 1] + cm[1, 0])     # TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=['fake', 'real']))
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

In [ ]:
import seaborn as sns

# Compute confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Define labels for the matrix
labels = ['Fake', 'Real']

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Predictions on the test dataset

predicted_probabilities = predictions.ravel()  # Flatten predictions to 1D array

# Convert labels from dataset to numpy array for evaluation
true_labels = np.array(test_labels)

# Compute ROC curve and AUC
fpr, tpr, thresholds = roc_curve(true_labels, predicted_probabilities)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', lw=1)  # Diagonal line
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Evaluate model performance on validation dataset
val_loss, val_accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
# image path
image_path = "/kaggle/input/images/real/0000.jpg"

# Load and preprocess the image using preprocess_image function
def load_and_preprocess_image(file_path):
    # Read the image file
    img = tf.io.read_file(file_path)
    # Decode the image to RGB format
    img = tf.image.decode_jpeg(img, channels=3)
    # Resize the image to match the input size for EfficientNetB4
    img = tf.image.resize(img, (380, 380))
    # Preprocess the image using EfficientNet preprocessing
    img = preprocess_input(img)
    return img

# Load and preprocess the image
input_image = load_and_preprocess_image(image_path)

# Reshape the image to a batch of 1 (since we're predicting a single image)
input_image = np.expand_dims(input_image, axis=0)

# Make prediction
prediction = model.predict(input_image)

# Display the raw predicted probabilities
print(f"Raw Prediction: {prediction}")

# Interpret the prediction
probability_fake = prediction[0][0]
probability_real = 1 - probability_fake

# Display the interpreted probabilities
print(f"Probability for 'fake' class: {probability_fake:.4f}")
print(f"Probability for 'real' class: {probability_real:.4f}")

predicted_class = 'fake' if probability_fake > 0.5 else 'real'
print(f"Predicted Class: {predicted_class}")

In [ ]:

saved_model_path = "Saved_Model_2"

# Save the model to the specified path

model.save(saved_model_path)

In [ ]:
import shutil

shutil.make_archive("/kaggle/working/Saved_Model_2", 'zip', "Saved_Model_2")


In [ ]:
# import os

# os.remove("/kaggle/working/Saved_Model_2.zip")
